# Technosphere flows induced by the REMIND electricity sector

Here we are interested in the flows of a specific (processed) material in the electricity sector in REMIND. **Values are given per kWh**.

In [70]:
%run initialize_notebook.ipynb

In [79]:
%run REMIND_Functions_to_modify_ecoinvent.ipynb

Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

In [27]:
%run EI2REMIND_Interface_Helpers.ipynb

In [2]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

In [65]:
def rmnd_technoflows(region, scenario="BAU", year=2030, tech_primers={}):
    """ Extract technosphere flows for the REMIND electricity sector in a REMIND region.
        A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
        REMIND technology x.
    """
    carma_techs = ['Biomass IGCC CCS', 'Biomass IGCC', 'Coal IGCC', 'Coal IGCC CCS', 'Coal PC CCS', 'Gas CCS']

    # note that Carma Database is linked to a specific version of ecoinvent!
    eidb_name = get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)
    carmadb_name = import_karma(eidb_name=eidb_name)
    carmadb = Database(carmadb_name)
    
    result = {}
    bio_names = []
    regions = ei_locations_in_remind_region(region)
    
    techno_markets = {
        "steel": [act for act in eidb if "market for steel," in act["name"] and
                "hot rolled" not in act["name"]],
        "concrete": [act for act in eidb if "market for concrete," in act["name"]],
        "copper": [act for act in eidb if "market for copper" == act["name"]],
        "aluminium": [act for act in eidb if "market for aluminium, primary" in act["name"] or
                    "market for aluminium alloy," in act["name"]],
        "gold": [act for act in eidb if "market for gold" in act["name"]]
    }
    
    print("Found following regions for {}: {}".format(region, regions))
    for key in available_electricity_generating_technologies:
        print("Processing RMND Technology {}".format(key))
        if key in tech_primers:
            techs = [tech_primers[key]]
        else:
            techs = available_electricity_generating_technologies[key]
        if len(techs) == 0:
            print("No technologies available for {}! Skipping.".format(key))
            continue
            
        units = {
            "kilowatt hour": 1.,
            "megajoule": 1./3.6
        }
        if key in carma_techs:
            lca = lca_for_multiple_techs_and_regions(techs, regions, carmadb,
                                                    units_and_conversions=units)
        else:
            lca = lca_for_multiple_techs_and_regions(techs, regions, eidb,
                                                    units_and_conversions=units)

        act_indeces = {
            key: [lca.activity_dict[act.key] for act in techno_markets[key]] for key in techno_markets
        }
            
        # flows are aggregated by the resp. material
        result[key] = pd.DataFrame.from_dict({
            "flow": list(techno_markets.keys()),
            "amount": [lca.supply_array[act_indeces[key]].sum() for key in techno_markets]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [81]:
df = rmnd_technoflows("EUR")

Database CarmaCCS_ecoinvent_Remind_BAU_2030 already present.
Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']
Processing RMND Technology Biomass IGCC CCS
Could not find any activities matching ['Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 200km, storage 1000m/2025', 'Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m/2025', 'Electricity, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m/2025']


AttributeError: 'NoneType' object has no attribute 'activity_dict'

In [36]:
eidb = Database("ecoinvent_Remind_BAU_2030")

In [37]:
eidb.get("52178137d5269104efab9937c4663e08")

'market for steel, chromium steel 18/8' (kilogram, GLO, None)

In [28]:
steel_activities = [act for act in eidb if "market" in act["name"] and
    "steel" in act["reference product"]]

In [30]:
# What are reference products?
set([act["reference product"] for act in steel_activities])

{'air distribution terminal panel, steel, 120 m3/h',
 'aluminium around steel bi-metal stranded cable, 3x3.67mm external diameter wire',
 'aluminium around steel bi-metal wire, 3.67mm external diameter',
 'building, hall, steel construction',
 'casting, steel, lost-wax',
 'chromium steel pipe',
 'chromium steel removed by drilling, computer numerical controlled',
 'chromium steel removed by drilling, conventional',
 'chromium steel removed by milling, average',
 'chromium steel removed by milling, dressing',
 'chromium steel removed by milling, large parts',
 'chromium steel removed by milling, small parts',
 'chromium steel removed by turning, average, computer numerical controlled',
 'chromium steel removed by turning, average, conventional',
 'chromium steel removed by turning, primarily dressing, computer numerical controlled',
 'chromium steel removed by turning, primarily dressing, conventional',
 'chromium steel removed by turning, primarily roughing, computer numerical controll

In [31]:
isics = [act for act in eidb if ('ISIC rev.4 ecoinvent',
   '2410:Manufacture of basic iron and steel') in act["classifications"]]

In [32]:
isics

['market for sheet rolling, chromium steel' (kilogram, GLO, None),
 'steel production, converter, chromium steel 18/8' (kilogram, RoW, None),
 'market for tin plated chromium steel sheet, 2 mm' (square meter, GLO, None),
 'market for steel, unalloyed' (kilogram, GLO, None),
 'sheet rolling, chromium steel' (kilogram, RER, None),
 'ferronickel production, 25% Ni' (kilogram, GLO, None),
 'forging, steel, large open die' (kilogram, RoW, None),
 'steel production, low-alloyed, hot rolled' (kilogram, RoW, None),
 'ferrosilicon production' (kilogram, RoW, None),
 'cast iron production' (kilogram, RoW, None),
 'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None),
 'market for hot rolling, steel' (kilogram, GLO, None),
 'market for sheet rolling, steel' (kilogram, GLO, None),
 'steel production, low-alloyed, hot rolled' (kilogram, RER, None),
 'market for cast iron' (kilogram, GLO, None),
 'market for drawing of pipe, steel' (kilogram, GLO, None),
 'market for forging, steel' (kil

In [19]:
a.as_dict()

{'comment': 'This market does not contain any transportation as the product is considered a service activity.',
 'classifications': [('ISIC rev.4 ecoinvent',
   '2410:Manufacture of basic iron and steel'),
  ('EcoSpold01Categories', 'metals/extraction')],
 'activity type': 'market activity',
 'activity': '82e61e26-5515-4843-bbd6-dd17d2663ff6',
 'database': 'ecoinvent_3.5',
 'filename': '24653_82e61e26-5515-4843-bbd6-dd17d2663ff6_8cb650d5-8a48-4719-81b1-2080b3b4387f.spold',
 'location': 'GLO',
 'name': 'market for steel, low-alloyed, hot rolled',
 'parameters': [],
 'authors': {'data entry': {'name': 'Guillaume Bourgault',
   'email': 'bourgault@ecoinvent.org'},
  'data generator': {'name': '[System]', 'email': 'support@ecoinvent.org'}},
 'type': 'process',
 'reference product': 'steel, low-alloyed, hot rolled',
 'flow': '8cb650d5-8a48-4719-81b1-2080b3b4387f',
 'unit': 'kilogram',
 'production amount': 1.0,
 'code': 'b5fb0d05c3219a322b99f65f92fd8d18'}

In [6]:
w = [act for act in eidb if "electricity production, wind" in act["name"]][0]

In [7]:
w

'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, CN-YN, None)

In [8]:
# What is used here?
lca = LCA({w: 1})
lca.lci()

In [9]:
tech_names = [get_activity(key)["name"] for key in lca.activity_dict]

In [14]:
df = pd.DataFrame.from_dict({
    "name": tech_names,
"key": list(lca.activity_dict.keys()),
"amount": lca.supply_array})

In [39]:
df.sort_values("amount", ascending=False)

,name,key,amount
8752,"electricity production, wind, >3MW turbine, on...","(ecoinvent_3.5, a130ca8bd1d4d0635f72f0b1305511ca)",1.000001e+00
13387,"market for sulfidic tailing, off-site","(ecoinvent_3.5, c52df2a767385ca1a9154c0dc9623b9e)",1.069800e-01
6074,"treatment of sulfidic tailing, off-site","(ecoinvent_3.5, 72eee5e2793526c3a0c37a3f98ffe144)",1.069800e-01
5189,"market for transport, freight, sea, transocean...","(ecoinvent_3.5, b6d5646c25516b5a9bee93872212bde8)",6.625315e-02
7353,"transport, freight, sea, transoceanic ship","(ecoinvent_3.5, 2a4fdfea31c4a622a3b1fb7efe6f9a7d)",6.625315e-02
14681,market for waste reinforced concrete,"(ecoinvent_3.5, e66ddfb72a8c383e1e1ab0a1de81e9b6)",6.253258e-02
9612,market for coke,"(ecoinvent_3.5, 5616d9d9e9c2633d9fbd362c38c40151)",5.846439e-02
11464,coking,"(ecoinvent_3.5, ecd0afd430c50b1fdb00c630d4e7523d)",5.800461e-02
7097,"market for process-specific burdens, inert mat...","(ecoinvent_3.5, 3ec25ecba84df3b3c8873a4ea4f28ac5)",3.946764e-02
2330,"process-specific burdens production, inert mat...","(ecoinvent_3.5, 77c2cb982d8dc1e4df79b1166a43fbb2)",3.946764e-02


Looks like most steel is distributed by the two markets for low- and unalloyed steel (reinforcing steel is made by low- and unalloyed steel).

In [28]:
[act for act in eidb if "market for steel," in act["name"]]

['market for steel, chromium steel 18/8, hot rolled' (kilogram, GLO, None),
 'market for steel, unalloyed' (kilogram, GLO, None),
 'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None),
 'market for steel, low-alloyed' (kilogram, GLO, None),
 'market for steel, chromium steel 18/8' (kilogram, GLO, None)]

In [31]:
steel_markets = [act for act in eidb if "market for steel," in act["name"] and
                "hot rolled" not in act["name"]]

In [32]:
steel_markets

['market for steel, low-alloyed' (kilogram, GLO, None),
 'market for steel, unalloyed' (kilogram, GLO, None),
 'market for steel, chromium steel 18/8' (kilogram, GLO, None)]

In [35]:
copper_markets = [act for act in eidb if "market for copper" == act["name"]]

In [36]:
copper_markets

['market for copper' (kilogram, GLO, None)]

In [37]:
gold_markets = [act for act in eidb if "market for gold" in act["name"]]

In [38]:
gold_markets

['market for gold-silver, ingot' (kilogram, GLO, None),
 'market for gold' (kilogram, GLO, None)]

In [44]:
concrete_markets = [act for act in eidb if "market for concrete," in act["name"]]

In [45]:
concrete_markets

['market for concrete, 20MPa' (cubic meter, GLO, None),
 'market for concrete, high exacting requirements' (cubic meter, RoW, None),
 'market for concrete, 35MPa' (cubic meter, GLO, None),
 'market for concrete, high exacting requirements' (cubic meter, CH, None),
 'market for concrete, sole plate and foundation' (cubic meter, RoW, None),
 'market for concrete, 50MPa' (cubic meter, GLO, None),
 'market for concrete, normal' (cubic meter, CH, None),
 'market for concrete, 30-32MPa' (cubic meter, GLO, None),
 'market for concrete, normal' (cubic meter, RoW, None),
 'market for concrete, 25MPa' (cubic meter, GLO, None),
 'market for concrete, for de-icing salt contact' (cubic meter, CH, None),
 'market for concrete, for de-icing salt contact' (cubic meter, RoW, None),
 'market for concrete, sole plate and foundation' (cubic meter, CH, None)]

In [49]:
aluminium_markets = [act for act in eidb if "market for aluminium, primary" in act["name"] or
                    "market for aluminium alloy," in act["name"]]

In [50]:
aluminium_markets

['market for aluminium alloy, AlLi' (kilogram, GLO, None),
 'market for aluminium, primary, liquid' (kilogram, GLO, None),
 'market for aluminium alloy, AlMg3' (kilogram, GLO, None),
 'market for aluminium, primary, cast alloy slab from continuous casting' (kilogram, GLO, None),
 'market for aluminium alloy, metal matrix composite' (kilogram, GLO, None),
 'market for aluminium, primary, ingot' (kilogram, RoW, None),
 'market for aluminium, primary, ingot' (kilogram, IAI Area, EU27 & EFTA, None)]

In [4]:
techno_markets = {
    "steel": [act for act in eidb if "market for steel," in act["name"] and
                "hot rolled" not in act["name"]],
    "concrete": [act for act in eidb if "market for concrete," in act["name"]],
    "copper": [act for act in eidb if "market for copper" == act["name"]],
    "aluminium": [act for act in eidb if "market for aluminium, primary" in act["name"] or
                    "market for aluminium alloy," in act["name"]],
    "gold": [act for act in eidb if "market for gold" in act["name"]]
}

In [6]:
lca=LCA({eidb.random(): 1})
lca.lci()

In [12]:
lca.supply_array[[1,3,5]]

array([0.        , 0.        , 0.00147585])